In [5]:
import argparse
from datetime import datetime
import json
import logging
import os
import math
import sys
import time


from gridappsd import GridAPPSD, DifferenceBuilder, utils, topics as t
from gridappsd.topics import simulation_input_topic, simulation_log_topic, simulation_output_topic, platform_log_topic


#model_mrid = sim_request["power_system_config"]["Line_name"]
#model_mrid = "_C125761E-9C21-4CA9-9271-B168150DE276" #ieee13training
#model_mrid = "_EE71F6C9-56F0-4167-A14E-7F4C71F10EAA" #final9500node
#model_mrid = "_AAE94E4A-2465-6F5E-37B1-3E72183A4E44" #test9500
model_mrid = "_49AD8E07-3BF9-A4E2-CB8F-C3722F837B62"  # 13 bus
#model_mrid = "_5B816B93-7A5F-B64C-8460-47C17D6E4B0F" #ieee13assets
#model_mrid="_C1C3E687-6FFD-C753-582B-632A27E28507" # IEEE 123
#model_mrid="_49AD8E07-3BF9-A4E2-CB8F-C3722F837B62"

requestedTime=1570041125



global BaseConnDict,BaseTermDict,TermList, NodeList
global XfmrKeys,XfmrDict,SwitchKeys,SwitchDict,DG_query

os.environ['GRIDAPPSD_APPLICATION_ID'] = 'gridappsd-topology-processor'
os.environ['GRIDAPPSD_APPLICATION_STATUS'] = 'STARTED'
#os.environ['GRIDAPPSD_SIMULATION_ID'] = opts.simulation_id
os.environ['GRIDAPPSD_USER'] = 'app_user'
os.environ['GRIDAPPSD_PASSWORD'] = '1234App'


gapps = GridAPPSD()
assert gapps.connected

In [4]:
import topologyprocessor, topologydictionary, networkmodel, distributedtopology

In [12]:
message = {
   "requestType": "GET_SWITCH_AREAS",
   "modelID": "ABC-1234MRID",
   "resultFormat": "JSON"
}

topic = "/queue/goss.gridappsd.request.data.topology"

gapps.send(topic,message)